<a href="https://colab.research.google.com/github/ujwalkr94/LLM_Developer/blob/main/notebooks/Cohere_and_Open_Source_Embedding_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install Packages and Setup Variables**

In [ ]:
!pip install -q llama-index==0.12.12 openai==1.59.6 llama-index-finetuning==0.3.0 llama-index-embeddings-huggingface==0.5.1 llama-index-embeddings-cohere==0.4.0 llama-index-readers-web==0.3.4 cohere==5.13.8 tiktoken==0.8.0 chromadb==0.6.3 sentence-transformers==2.7.0 pydantic==2.10.5 llama-index-vector-stores-chroma==0.4.1 llama-index-llms-gemini==0.4.4 llama-index-embeddings-instructor==0.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 95.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 46.4 

In [ ]:
import os

# Set the following API Keys in the Python environment. Will be used later.
os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_KEY>"
os.environ["CO_API_KEY"] = "<YOUR_OPENAI_KEY>"

# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY1')
# os.environ["CO_API_KEY"] = userdata.get('CO_API_KEY')

In [ ]:
# Allows running asyncio in environments with an existing event loop, like Jupyter notebooks.

import nest_asyncio

nest_asyncio.apply()

**Load a Model**

In [ ]:
from llama_index.core import Settings
# For LLM

from llama_index.llms.openai import OpenAI
Settings.llm = OpenAI(temperature=1, model="gpt-4o-mini")

**Load the Dataset (CSV)**

Download


The dataset includes several articles from the TowardsAI blog, which provide an in-depth explanation of the LLaMA2 model. Read the dataset as a long string.

In [ ]:
!curl -o ./mini-llama-articles.csv https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  169k  100  169k    0     0   248k      0 --:--:-- --:--:-- --:--:--  248k


**Read File**

In [ ]:
import csv

rows = []

# Load the file as a JSON
with open("./mini-llama-articles.csv", mode="r", encoding="utf-8") as file:
    csv_reader = csv.reader(file)

    for idx, row in enumerate(csv_reader):
        if idx == 0:
            continue
            # Skip header row
        rows.append(row)

# The number of characters in the dataset.
len(rows)

14

**Convert to Document obj**

In [ ]:
from llama_index.core import Document

# Convert the chunks to Document objects so the LlamaIndex framework can process them.
documents = [
    Document(
        text=row[1], metadata={"title": row[0], "url": row[2], "source_name": row[3]}
    )
    for row in rows
]

# Cohere Embedding Model

### Creating Vector Store

In [ ]:
import chromadb

# create a client and a new collection
# chromadb.EphemeralClient saves data in-memory.

chroma_client = chromadb.PersistentClient(path="./mini-llama-articles")
chroma_collection = chroma_client.create_collection("mini-llama-articles")

In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore

# Define a storage context object using the created vector database.
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

**Transforming**

In [ ]:
from llama_index.core.text_splitter import TokenTextSplitter

# Define the splitter object that split the text into segments with 512 tokens,
# with a 128 overlap between the segments.
text_splitter = TokenTextSplitter(separator=" ", chunk_size=512, chunk_overlap=128)

There are two options to use the Cohere embeddings:

- input_type="search_document": Employ this option for texts (documents) intended for storage in your vector database.

- input_type="search_query": Use this when issuing search queries to locate the most related documents within your vector database.

In [ ]:
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    KeywordExtractor,
)
from llama_index.embeddings.cohere import CohereEmbedding
from llama_index.core.ingestion import IngestionPipeline


from llama_index.llms.openai import OpenAI
llm_gpt_4o = OpenAI(temperature=0, model="gpt-4o-mini")


# Create the pipeline to apply the transformation on each chunk,
# and store the transformed text in the chroma vector store.
pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
        QuestionsAnsweredExtractor(questions=3, llm=llm_gpt_4o),
        SummaryExtractor(summaries=["prev", "self"], llm=llm_gpt_4o),
        KeywordExtractor(keywords=10, llm=llm_gpt_4o),
        CohereEmbedding(model_name="embed-english-v3.0", input_type="search_document", cohere_api_key= 'CO_API_KEY'),
    ],
    vector_store=vector_store,
)

# Run the transformation pipeline.
nodes = pipeline.run(documents=documents, show_progress=True)

Parsing nodes:   0%|          | 0/14 [00:00<?, ?it/s]

100%|██████████| 108/108 [01:30<00:00,  1.19it/s]


Generating embeddings:   0%|          | 0/108 [00:00<?, ?it/s]

In [ ]:
len(nodes)

108

In [ ]:
len(nodes[0].embedding)

1024

In [ ]:
# Compress the vector store directory to a zip file to be able to download and use later.
!zip -r vectorstore_cohere.zip mini-llama-articles

  adding: mini-llama-articles/ (stored 0%)
  adding: mini-llama-articles/242dc8b8-3cda-46ae-95fd-279406c1a093/ (stored 0%)
  adding: mini-llama-articles/242dc8b8-3cda-46ae-95fd-279406c1a093/link_lists.bin (stored 0%)
  adding: mini-llama-articles/242dc8b8-3cda-46ae-95fd-279406c1a093/header.bin (deflated 61%)
  adding: mini-llama-articles/242dc8b8-3cda-46ae-95fd-279406c1a093/data_level0.bin (deflated 100%)
  adding: mini-llama-articles/242dc8b8-3cda-46ae-95fd-279406c1a093/length.bin (deflated 59%)
  adding: mini-llama-articles/chroma.sqlite3 (deflated 80%)


**Load Indexes**

If you have already uploaded the zip file for the vector store checkpoint, please uncomment the code in the following cell block to extract its contents. After doing so, you will be able to load the dataset from local storage.

In [ ]:
# Load the vector store from the local storage.
db = chromadb.PersistentClient(path="./mini-llama-articles")
chroma_collection = db.get_or_create_collection("mini-llama-articles")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [ ]:
from llama_index.core import VectorStoreIndex

# Define the Cohere Embedding Model
embed_model = CohereEmbedding( model_name="embed-english-v3.0", input_type="search_query", cohere_api_key= userdata.get('CO_API_KEY'))

# Create the index based on the vector store.
index = VectorStoreIndex.from_vector_store( vector_store, embed_model = embed_model)

**Query Dataset**

In [ ]:
# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.
query_engine = index.as_query_engine(similarity_top_k=5)

res = query_engine.query("How many parameters LLaMA2 model has?")

In [ ]:
res.response

'The Llama 2 model is available in four sizes, with the parameters being 7B (7 billion), 13B (13 billion), 34B (34 billion), and 70B (70 billion).'

In [ ]:
# Show the retrieved nodes
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Title\t", src.metadata["title"])
    print("Text\t", src.text)
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 2eeb2fb5-519b-4686-bf5d-c9951e5add21
Title	 Meta's Llama 2: Revolutionizing Open Source Language Models for Commercial Use
Text	 The model demonstrates exceptionally low AI safety violation percentages, surpassing even ChatGPT in safety benchmarks. Finding the right balance between helpfulness and safety when optimizing a model poses significant challenges. While a highly helpful model may be capable of answering any question, including sensitive ones like "How do I build a bomb?", it also raises concerns about potential misuse. Thus, striking the perfect equilibrium between providing useful information and ensuring safety is paramount. However, prioritizing safety to an extreme extent can lead to a model that struggles to effectively address a diverse range of questions. This limitation could hinder the model's practical applicability and user experience. Thus, achieving an optimum balance that allows the model to be both helpful and safe is of utmost importance. To strike th

# Open Source Embedding

In [ ]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

# Open Source Embedding Model Collection

chroma_client_ose = chromadb.PersistentClient(path="./mini-llama-articles-open-source-embed")
chroma_collection_ose = chroma_client_ose.create_collection("mini-llama-articles-open-source-embed")

# Define a storage context object using the created vector database.
vector_store_ose = ChromaVectorStore(chroma_collection=chroma_collection_ose)


In [ ]:
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    KeywordExtractor,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.ingestion import IngestionPipeline

from llama_index.llms.openai import OpenAI
llm_gpt_4o = OpenAI(temperature=0, model="gpt-4o-mini")

# Create the pipeline to apply the transformation on each chunk,
# and store the transformed text in the chroma vector store.
pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
        QuestionsAnsweredExtractor(questions=3, llm=llm_gpt_4o),
        SummaryExtractor(summaries=["prev", "self"], llm=llm_gpt_4o),
        KeywordExtractor(keywords=10, llm=llm_gpt_4o),
        HuggingFaceEmbedding(model_name="intfloat/e5-small-v2"),
    ],
    vector_store=vector_store_ose,
)

# Run the transformation pipeline.
nodes_ose = pipeline.run(documents=documents, show_progress=True)

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Parsing nodes:   0%|          | 0/14 [00:00<?, ?it/s]

100%|██████████| 108/108 [00:31<00:00,  3.47it/s]


Generating embeddings:   0%|          | 0/108 [00:00<?, ?it/s]

In [ ]:
print(len(nodes_ose))
print(len(nodes_ose[0].embedding))

108
384


In [ ]:
# Compress the vector store directory to a zip file to be able to download and use later.
!zip -r vectorstore_open_source_embed.zip mini-llama-articles-open-source-embed

  adding: mini-llama-articles-open-source-embed/ (stored 0%)
  adding: mini-llama-articles-open-source-embed/2a46f0c2-3aaa-4a95-9239-2371ffd5571f/ (stored 0%)
  adding: mini-llama-articles-open-source-embed/2a46f0c2-3aaa-4a95-9239-2371ffd5571f/link_lists.bin (stored 0%)
  adding: mini-llama-articles-open-source-embed/2a46f0c2-3aaa-4a95-9239-2371ffd5571f/header.bin (deflated 61%)
  adding: mini-llama-articles-open-source-embed/2a46f0c2-3aaa-4a95-9239-2371ffd5571f/data_level0.bin (deflated 100%)
  adding: mini-llama-articles-open-source-embed/2a46f0c2-3aaa-4a95-9239-2371ffd5571f/length.bin (deflated 58%)
  adding: mini-llama-articles-open-source-embed/chroma.sqlite3 (deflated 81%)


In [ ]:
# Load the vector store from the local storage.
db_ose = chromadb.PersistentClient(path="./mini-llama-articles-open-source-embed")
chroma_collection_ose = db_ose.get_or_create_collection("mini-llama-articles-open-source-embed")
vector_store_ose = ChromaVectorStore(chroma_collection=chroma_collection_ose)

In [ ]:
from llama_index.core import VectorStoreIndex

# Define the Cohere Embedding Model
embed_model_e5 = HuggingFaceEmbedding(model_name="intfloat/e5-small-v2")

# Create the index based on the vector store.
index_ose = VectorStoreIndex.from_vector_store( vector_store_ose, embed_model = embed_model_e5)

# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.
query_engine_ose = index_ose.as_query_engine(similarity_top_k=5)

res = query_engine_ose.query("How many parameters LLaMA2 model has?")

res.response

'Llama 2 is available in four different model sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters.'

In [ ]:
# Show the retrieved nodes
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Title\t", src.metadata["title"])
    print("Text\t", src.text)
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 c006a0c5-da1b-4727-b5e0-6b7ed23de54a
Title	 Meta's Llama 2: Revolutionizing Open Source Language Models for Commercial Use
Text	 I. Llama 2: Revolutionizing Commercial Use Unlike its predecessor Llama 1, which was limited to research use, Llama 2 represents a major advancement as an open-source commercial model. Businesses can now integrate Llama 2 into products to create AI-powered applications. Availability on Azure and AWS facilitates fine-tuning and adoption. However, restrictions apply to prevent exploitation. Companies with over 700 million active daily users cannot use Llama 2. Additionally, its output cannot be used to improve other language models.  II. Llama 2 Model Flavors Llama 2 is available in four different model sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters. While 7B, 13B, and 70B have already been released, the 34B model is still awaited. The pretrained variant, trained on a whopping 2 trillion tokens, boasts a context window of 4096 toke